In [7]:
# import required modules
import cv2
import math
import argparse
# Create function for age and gender detection
def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections=net.forward()
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn,faceBoxes


parser=argparse.ArgumentParser()
parser.add_argument('image1.jpg')

args=parser.parse_args()

# import data from the pre-trained dataset model to the variables for use in this module
faceProto="modelNweight/opencv_face_detector.pbtxt"
faceModel="modelNweight/opencv_face_detector_uint8.pb"
ageProto="modelNweight/age_deploy.prototxt"
ageModel="modelNweight/age_net.caffemodel"
genderProto="modelNweight/gender_deploy.prototxt"
genderModel="modelNweight/gender_net.caffemodel"

# Mentioned values which are used in pre-trained dataset
MODEL_MEAN_VALUES=(78.4263377603, 87.7689143744, 114.895847746)
ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList=['Male','Female']

faceNet=cv2.dnn.readNet(faceModel,faceProto)
ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto)
# Create to predict the Age and Gender while using web cam
video=cv2.VideoCapture(args.image if args.image else 0)
padding=20
while cv2.waitKey(1)<0 :
    hasFrame,frame=video.read()
    if not hasFrame:
        cv2.waitKey()
        break

    resultImg,faceBoxes=highlightFace(faceNet,frame)
    if not faceBoxes:
        print("No face detected")

    for faceBox in faceBoxes:
        face=frame[max(0,faceBox[1]-padding):
                   min(faceBox[3]+padding,frame.shape[0]-1),max(0,faceBox[0]-padding)
                   :min(faceBox[2]+padding, frame.shape[1]-1)]

        blob=cv2.dnn.blobFromImage(face, 1.0, (227,227), MODEL_MEAN_VALUES, swapRB=False)
        genderNet.setInput(blob)
        genderPreds=genderNet.forward()
        gender=genderList[genderPreds[0].argmax()]
        print(f'Gender: {gender}')

        ageNet.setInput(blob)
        agePreds=ageNet.forward()
        age=ageList[agePreds[0].argmax()]
        print(f'Age: {age[1:-1]} years')

        cv2.putText(resultImg, f'{gender}, {age}', (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2, cv2.LINE_AA)
        cv2.imshow("Detecting age and gender", resultImg)

usage: ipykernel_launcher.py [-h] image1.jpg
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

In [9]:
# Testing age and gender prediction of image1
Input = cv2.imread("assets/image1.jpg")
Output = Age_Gender_Detection(Input)
cv2.imshow("assets",Output)

NameError: name 'Age_Gender_Detection' is not defined

In [ ]:
# Testing age and gender prediction of image2
Input = cv2.imread("assets/image2.jpg")
Output = Age_Gender_Detection(Input)
cv2.imshow("assets",Output)

In [ ]:
# Testing age and gender prediction of image3
Input = cv2.imread("assets/image3.jpg")
Output = Age_Gender_Detection(Input)
cv2.imshow("assets",Output)

In [ ]:
# Testing age and gender prediction of image3
Input = cv2.imread("assets/image4.jpg")
Output = Age_Gender_Detection(Input)
cv2.imshow("assets",Output)
